# Connect

## Google Drive

The datasets containing the examples of Linguistic Antipattern that the model will have to be able to identify are located in a shared folder, created specifically for the project, on google drive, to facilitate data recovery operations they are then accessed to Drive, so you can then easily retrieve the datasets

In [1]:
# Mount in the Colab runtime a folder corresponding to your google drive

from google.colab import drive
drive.mount('/content/drive');

Mounted at /content/drive


## Hugging Face

We log in to Hugging Face so that we can upload the model lastly after finishing the operations necessary for Fine Tuning

In [ ]:
!pip install huggingface

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Installation

## Installing Tranformers Libraries

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 124.3 MB/s eta 0:00:00


## Installing the Necessary Libraries

In the next step we go instead to download all those libraries necessary to be able to carry out training, evaluation and testing of the model, also adding some libraries used to improve data visualization:

1.   **train_test_split**: this function comes from the _sklearn_ library, and allows you to divide a dataset into two parts: _training-set_ and _test-set_. Respectively, one will be used to train the model, while the other will be used to evaluate its generalization ability.
2.   **pandas**: offers tools for analyzing data in tabular form, dataframes and manipulating them. It allows operations such as column filtering, aggregation and merging of dataframes.
3.  **numpy**: processing of multidimensional numerical arrays, for the latter, offers linear algebra operations and mathematical operations.
4.  **tabulate**: allows you to transform an array into a table to graphically display data structures.
5.  **tqdm**: allows displaying progressbar in iterative processing loops. It could be useful at this stage of system development, to monitor the time required by the various processes.

In [3]:
import torch # ML framework
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler # library for data manipulation

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random # random numbers generation

import warnings
warnings.filterwarnings(action='once')

# Data Preparation

## Retrive Data from Dataset

in the code below, we get the data files from the google drive of the project

In [4]:
# Da eseguire se hai caricato il dataset a mano nel runtime
mop_instances = "/content/drive/Shareddrives/se4ai/mop.csv"
aop_instances = "/content/drive/Shareddrives/se4ai/aop.csv"
nic_instances = "/content/drive/Shareddrives/se4ai/nic.csv"

The training of the model will be carried out through supervised learning, i.e. labels are added to the data which will allow the model to understand what type of information it is analyzing, the labels are then transformed into numbers, so as to be able to be more easily included by the model.
The model must be able to recognize 4 different types of linguistic antipattern within python code, they are:
- **"Get" More Than an Accessor**: A getter that performs actions other than returning the corresponding attribute.
- **Not Implemented Condition**: The comments of a method suggest a conditional behavior that is not implemented in the code. When the implementation is default this should be documented.
- **Attribute Signature and Comment are Opposite**: The declaration of an attribute is in contradiction with its documentation.
- **Method Signature and Comment are Opposite**: The  declaration of a method is in contradiction with its documentation.

In addition to the 4 types of Antipattern, the model must also be able to recognize the instances of code that do not contain them, thus classifying them as instances of clean code (**Clear**)

The labels assigned to the various code snippets, based on the antipattern they contain, within the dataset are:
- **mop**: used for "Metohd Signature and Comment are Opposite", this label will be assigned the value **0** for model training
- **aop**: used for "Attribute Signature and Comment are Opposite", this label will be assigned the value **1** for model training
- **clr**: used for "Clear", this label will be assigned the value **2** for model training
- **nic**: used for "Not Implemented Condition", this label will be assigned the value **3** for model training
- **get**: used for "Get" More than an Accessor, this label will be assigned the value **4** for model training


In [5]:
mop = pd.read_csv(mop_instances) # pandas read_csv legge automaticamente file csv e crea un "oggetto"
aop = pd.read_csv(aop_instances)
nic = pd.read_csv(nic_instances)

# Creiamo array di label e array di snippet di codice
mop_labels = list([])
aop_labels = list([])
nic_labels = list([])
for l in list(mop['label']):
  if l == 'mop':
    mop_labels.append(0)
  elif l == 'clr':
    mop_labels.append(2)

for l in list(aop['label']):
  if l == 'aop':
    aop_labels.append(1)
  elif l == 'clr':
    aop_labels.append(2)

for l in list(nic['label']):
  if l == 'nic':
    nic_labels.append(3)
  elif l == 'clr':
    nic_labels.append(2)



mop_code = list(mop['code'])
aop_code = list(aop['code'])
nic_code = list(nic['code'])

print(mop_labels)
print(aop_labels)
print(nic_labels)

<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

## Split data for Training and Testing

In [6]:
from sklearn.model_selection import train_test_split # funzione in sklearn per dividere il dataset in train, test, validation
# Divide il dataset in train test validation
# Parametri test_size:
# valori di default: train test_size = 0.33 val test_size = 0.3
TRAIN_TEST_SIZE = 0.33;
VAL_TEST_SIZE = 0.3;
# Tendenzialmente dovrebbero essere simili così train = 70%, test = 30%, val = 30% di test;
# Alzando test e val può migliorare accuracy perché ha più esempi per test e validazione

mop_train_codes, mop_temp_codes, mop_train_labels, mop_temp_labels = train_test_split(mop_code, mop_labels, test_size = TRAIN_TEST_SIZE, shuffle = True, stratify = mop_labels);
aop_train_codes, aop_temp_codes, aop_train_labels, aop_temp_labels = train_test_split(aop_code, aop_labels, test_size = TRAIN_TEST_SIZE, shuffle = True, stratify = aop_labels);
nic_train_codes, nic_temp_codes, nic_train_labels, nic_temp_labels = train_test_split(nic_code, nic_labels, test_size = TRAIN_TEST_SIZE, shuffle = True, stratify = nic_labels);

train_codes = mop_train_codes + aop_train_codes + nic_train_codes
temp_codes = mop_temp_codes +  aop_temp_codes + nic_temp_codes
train_labels = mop_train_labels + aop_train_labels + nic_train_labels
temp_labels = mop_temp_labels + aop_temp_labels + nic_temp_labels

test_codes, val_codes, test_labels, val_labels = train_test_split(temp_codes, temp_labels, test_size = VAL_TEST_SIZE, shuffle = True, stratify = temp_labels );

print(train_codes)
print(train_labels)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:

['def load_tf_weights_in_bert(model, config, tf_checkpoint_path):\r\n\t"""\r\n\t\tLoad tf checkpoints in a pytorch model.\r\n\t"""', 'def calculate_sum(a, b):\r\n\t"""\r\n\t\tThis function adds the two numbers and returns the result.\r\n\t"""', 'def delete_file(file_path, file_name):\r\n\t"""\r\n\t\tcreates a text file within the specified system path, \r\n\t\tif the path is valid and there is no other file with the same name and extension, the file is created, \r\n\t\totherwise an exception is thrown\r\n\t\tArgs:\r\n\t\t\tfile_path: the destination folder in which to create the file\r\n\t\t\tfile_name: the name of the file to create\r\n\t"""', 'def display():\r\n\t"""\r\n\t\tshows the image stored inside the instance variable of the class\r\n\t"""', 'def create_object(my_obj):\r\n\t"""\r\n\t\tgiven the instance of a certain object, this function destroys it, \r\n\t\tin order to do this, the class must be a child of the "ObjectUtils" custom class,\r\n\t\totherwise the dedicated method 

<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


# Model Evaluations Metrics



## Metrics Calculation

Let's now import the functions that calculate the metrics of our model, through which we will be able to understand how efficient it is in determining whether an instance of code to be analyzed belongs to a certain class or to another.
The Metrics used to evaluate our model are:

- **Accuracy**: It gives you the overall accuracy of the model, meaning the fraction of the total samples that were correctly classified by the classifier. To calculate accuracy, use the following formula: (TP+TN)/(TP+TN+FP+FN).
- **Precision**: It tells you what fraction of predictions as a positive class were actually positive. To calculate precision, use the following formula: TP/(TP+FP).
- **Recall**: It tells you what fraction of all positive samples were correctly predicted as positive by the classifier. It is also known as True Positive Rate (TPR), Sensitivity, Probability of Detection. To calculate Recall, use the following formula: TP/(TP+FN).
- **Specificity**: It tells you what fraction of all negative samples are correctly predicted as negative by the classifier. It is also known as True Negative Rate (TNR). To calculate specificity, use the following formula: TN/(TN+FP).
- **F1 Score**: It combines precision and recall into a single measure. Mathematically it’s the harmonic mean of precision and recall.

installiamo la libreria **evaluate** di hugging face



In [7]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.2 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/google/colab/_pip.py:85: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.10/dist-packages/aiohttp-3.8.4.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):
/usr/local/lib/python3.10/dist-packages/google/colab/_pip.py:85: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.10/dist-packages/aiosignal-1.3.1.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):
/usr/local/lib/python3.10/dist-packages/google/colab/_pip.py:85: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.10/dist-packages/async_timeout-4.0.2.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):
/usr/local/lib/python3.10/dist-packages/google/colab/_pip.py:85: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.10/dist-packages/datasets-2.12.0.dist-info/top_level.txt' mode='r' encoding='

We import the metrics with an avarage of type "macro".
This step is very important as the evaluation metrics of this library are default for binary classification problems, in our case we need to set them in such a way that we have some overall evaluation metrics to evaluate the efficiency for a classification model multiclass, and this is done by setting the average to "macro"

In [8]:
import evaluate

accuracy = evaluate.load("accuracy", average="macro")
precision = evaluate.load("precision", average="macro")
f1 = evaluate.load("f1", average="macro")
recall = evaluate.load("recall", average="macro")
mcc = evaluate.load("matthews_correlation", avarage="macro")

<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen imp

# Model Inizialization

## Roberta Tokenizer

We download the HuggingFace tokenization module which selects the tokenizer of the ML model **Roberta**, as it is the model on which CodeBERT is based.
It splits text into tokens using the WordPiece algorithm, which breaks words into smaller, more common parts (subwords), so you can handle unknown or infrequent words during training.

In [9]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base', do_lower_case = True);

## Roberta for Sequesnce Classification

We download the module through which it is possible to select the Machine Learning models that are based on the **Roberta** model, as in our case for codeBERT.

With this specific module the model is loaded by adding a linear level above the **pooled output**, which is nothing more than an attention head for sequence classification/regression.
CodeBERT is in fact a model that allows you to perform feature extraction operations, while our model must be able to perform code classification operations, hence the need to train it to perform an operation different from its base, and therefore to add a new attention head.

In [10]:
from transformers import RobertaForSequenceClassification
# Quando esegui per la prima volta (nello stesso runtime) li deve scaricare.
# Dopo aver eseguito returna questo warning:
# ---
# Some weights of the model checkpoint at microsoft/codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
# This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
# ---
# Noi stiamo facendo esattamente quello che dice quindi va bene.
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base', do_lower_case = True);

id2label = {0: "mop", 2: "clr", 1: "aop", 3: "nic"}
label2id = {"mop": 0, "clr": 2, "aop": 1, "nic": 3}

# Attenzione al parametro num_labels in base a quante label si devono classificare (4 = aop, clr, mop, nic)
model = RobertaForSequenceClassification.from_pretrained('microsoft/codebert-base', num_labels = 4, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at microsoft/codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be 

In [11]:
def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 90,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )

def preprocessing_batch(data_set):
    token_id = []
    attention_masks = []
    for sample in data_set:
      encoding_dict = preprocessing(sample, tokenizer)
      token_id.append(encoding_dict['input_ids']) 
      attention_masks.append(encoding_dict['attention_mask'])
    token_id = torch.cat(token_id, dim = 0)
    attention_masks = torch.cat(attention_masks, dim = 0)
    return token_id,attention_masks;

train_token_id,train_attention_masks = preprocessing_batch(train_codes);
test_token_id,test_attention_masks = preprocessing_batch(test_codes);
val_token_id,val_attention_masks = preprocessing_batch(val_codes);

def print_rand_sentence_encoding(text, token_id):
  '''Displays tokens, token IDs and attention mask of a random text sample'''
  index = random.randint(0, len(text) - 1)
  tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
  token_ids = [i.numpy() for i in token_id[index]]
  print(tokens);
  table = np.array([tokens, token_ids]).T
  print(tabulate(table, 
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


We take the labels of our dataset that we have divided into different samples for training, testing and validation and transform them into Tensors

In [12]:
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)
val_labels = torch.tensor(val_labels)

In [13]:
batch_size = 16

train_set = TensorDataset(train_token_id, 
                          train_attention_masks, 
                          train_labels)

val_set = TensorDataset(val_token_id, 
                        val_attention_masks, 
                        val_labels)

test_set = TensorDataset(test_token_id, 
                        test_attention_masks, 
                        test_labels)

train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )


test_dataloader = DataLoader(
            test_set,
            sampler = SequentialSampler(test_set),
            batch_size = batch_size
        )

# Optimization

Let's optimize the parameters of our model in order to obtain the best possible result for the fine tuning, specifically let's modify:
1. **Learning Rate (lr)**: a hyperparameter that controls how much to change the model in response to the estimated error each time the model weights are updated. In our case it was set to 5e-5
2. **Epsilon**: used as a guard against a by zero division. In our case it was set to 1e-08
3. **Weight Decay**: a regularization technique by adding a small penalty, usually the L2 norm of the weights (all the weights of the model), to the loss function. In our case it was set to 0.01

In [14]:
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5, # LEARNING RATE DELL'ALGORITMO OTTIMIZZATORE (2e-5 = 2*10^-5 = 0.00005)
                              eps = 1e-08,
                              weight_decay = 0.01
                              )
model.cuda(); # Eseguire per impostare il modello in modo da usare la GPU durante training

# Training

The training of the model is carried out in this block, it has been set with a batch size of 16 and 8 epochs, for each batch of each epoch the metrics calculated for that training are printed on the screen.
At the end of the Training we will have a model developed to carry out classification operations, and capable of recognizing the linguistic antipatterns within the python code described above.

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

epochs = 8

for _ in trange (epochs, desc = 'Epoch'):
  model.train()
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0

  for step, batch in enumerate(train_dataloader):
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_labels = batch
      #reset gradient value for the new epoch
      optimizer.zero_grad()
      # Forward pass
      train_output = model(b_input_ids, 
                            token_type_ids = None, 
                            attention_mask = b_input_mask, 
                            labels = b_labels)
      # Backward pass
      train_output.loss.backward()
      optimizer.step()
      # Update tracking variables
      tr_loss += train_output.loss.item()
      nb_tr_examples += b_input_ids.size(0)
      nb_tr_steps += 1

       # ========== Validation ==========

      # Set model to evaluation mode
      model.eval()

      # Tracking variables 
      val_accuracy = []
      val_precision = []
      val_recall = []
      val_specificity = []

      latest_acc = 0.0000;

      for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics for the entire model
        predictions = np.argmax(logits, axis=1)
        print('\n\t - Train loss: {:.5f}'.format(tr_loss / nb_tr_steps))
        print('\t - Accuracy: {:.4f}'.format(accuracy.compute(predictions=predictions, references=label_ids)['accuracy']))
        print('\t - F1 Score: {:.4f}'.format(f1.compute(predictions=predictions, references=label_ids, average="macro")['f1']))
        print('\t - Precision: {:.4f}'.format(precision.compute(predictions=predictions, references=label_ids, average="macro", zero_division=0)['precision']))
        print('\t - Recall: {:.4f}'.format(recall.compute(predictions=predictions, references=label_ids, average="macro", zero_division=0)['recall']))
        print('\t - MCC: {:.4f}'.format(mcc.compute(predictions=predictions, references=label_ids, average="macro")['matthews_correlation']))


PATH = './greet'
torch.save(model, PATH)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Epoch:   0%|          | 0/8 [00:00<?, ?it/s]


	 - Train loss: 1.23388
	 - Accuracy: 0.4375
	 - F1 Score: 0.1522
	 - Precision: 0.1094
	 - Recall: 0.2500
	 - MCC: 0.0000

	 - Train loss: 1.23388
	 - Accuracy: 0.4375
	 - F1 Score: 0.1522
	 - Precision: 0.1094
	 - Recall: 0.2500
	 - MCC: 0.0000

	 - Train loss: 1.23388
	 - Accuracy: 0.5625
	 - F1 Score: 0.1800
	 - Precision: 0.1406
	 - Recall: 0.2500
	 - MCC: 0.0000

	 - Train loss: 1.23388
	 - Accuracy: 0.5625
	 - F1 Score: 0.1800
	 - Precision: 0.1406
	 - Recall: 0.2500
	 - MCC: 0.0000

	 - Train loss: 1.23388
	 - Accuracy: 0.4375
	 - F1 Score: 0.1522
	 - Precision: 0.1094
	 - Recall: 0.2500
	 - MCC: 0.0000

	 - Train loss: 1.23388
	 - Accuracy: 0.6667
	 - F1 Score: 0.2667
	 - Precision: 0.2222
	 - Recall: 0.3333
	 - MCC: 0.0000

	 - Train loss: 1.28430
	 - Accuracy: 0.4375
	 - F1 Score: 0.1522
	 - Precision: 0.1094
	 - Recall: 0.2500
	 - MCC: 0.0000

	 - Train loss: 1.28430
	 - Accuracy: 0.4375
	 - F1 Score: 0.1522
	 - Precision: 0.1094
	 - Recall: 0.2500
	 - MCC: 0.0000

	 - Tra

Epoch:  12%|█▎        | 1/8 [00:34<04:02, 34.69s/it]


	 - Train loss: 0.92400
	 - Accuracy: 0.5625
	 - F1 Score: 0.5904
	 - Precision: 0.6280
	 - Recall: 0.7500
	 - MCC: 0.4681

	 - Train loss: 0.92400
	 - Accuracy: 0.5000
	 - F1 Score: 0.5486
	 - Precision: 0.5000
	 - Recall: 0.6607
	 - MCC: 0.3608

	 - Train loss: 0.92400
	 - Accuracy: 0.2222
	 - F1 Score: 0.1429
	 - Precision: 0.1000
	 - Recall: 0.2500
	 - MCC: 0.0224

	 - Train loss: 0.73511
	 - Accuracy: 0.5625
	 - F1 Score: 0.5048
	 - Precision: 0.3896
	 - Recall: 0.7500
	 - MCC: 0.5217

	 - Train loss: 0.73511
	 - Accuracy: 0.5625
	 - F1 Score: 0.6627
	 - Precision: 0.6736
	 - Recall: 0.7357
	 - MCC: 0.4424

	 - Train loss: 0.73511
	 - Accuracy: 0.3750
	 - F1 Score: 0.3111
	 - Precision: 0.2381
	 - Recall: 0.5000
	 - MCC: 0.2940

	 - Train loss: 0.73511
	 - Accuracy: 0.5625
	 - F1 Score: 0.5904
	 - Precision: 0.6280
	 - Recall: 0.7500
	 - MCC: 0.4681

	 - Train loss: 0.73511
	 - Accuracy: 0.5000
	 - F1 Score: 0.5486
	 - Precision: 0.5000
	 - Recall: 0.6607
	 - MCC: 0.3608

	 - Tra

Epoch:  25%|██▌       | 2/8 [01:03<03:08, 31.36s/it]

...


	 - Train loss: 0.21722
	 - Accuracy: 0.4444
	 - F1 Score: 0.2051
	 - Precision: 0.1905
	 - Recall: 0.2222
	 - MCC: -0.2988

	 - Train loss: 0.03449
	 - Accuracy: 0.8125
	 - F1 Score: 0.8042
	 - Precision: 0.8125
	 - Recall: 0.8643
	 - MCC: 0.7369

	 - Train loss: 0.03449
	 - Accuracy: 0.6250
	 - F1 Score: 0.7083
	 - Precision: 0.7095
	 - Recall: 0.7238
	 - MCC: 0.4651

	 - Train loss: 0.03449
	 - Accuracy: 0.6875
	 - F1 Score: 0.5271
	 - Precision: 0.5083
	 - Recall: 0.5694
	 - MCC: 0.4665

	 - Train loss: 0.03449
	 - Accuracy: 0.6250
	 - F1 Score: 0.5938
	 - Precision: 0.6619
	 - Recall: 0.7222
	 - MCC: 0.4640

	 - Train loss: 0.03449
	 - Accuracy: 0.6250
	 - F1 Score: 0.6146
	 - Precision: 0.6806
	 - Recall: 0.5952
	 - MCC: 0.4532

	 - Train loss: 0.03449
	 - Accuracy: 0.4444
	 - F1 Score: 0.2051
	 - Precision: 0.1905
	 - Recall: 0.2222
	 - MCC: -0.2988

	 - Train loss: 0.08634
	 - Accuracy: 0.7500
	 - F1 Score: 0.7470
	 - Precision: 0.7917
	 - Recall: 0.8143
	 - MCC: 0.6551

	 - T

Epoch:  88%|████████▊ | 7/8 [03:33<00:30, 30.11s/it]


	 - Train loss: 0.13770
	 - Accuracy: 0.5000
	 - F1 Score: 0.5071
	 - Precision: 0.5905
	 - Recall: 0.6667
	 - MCC: 0.3419

	 - Train loss: 0.13770
	 - Accuracy: 0.7500
	 - F1 Score: 0.7595
	 - Precision: 0.7827
	 - Recall: 0.8036
	 - MCC: 0.6573

	 - Train loss: 0.13770
	 - Accuracy: 0.6667
	 - F1 Score: 0.4231
	 - Precision: 0.4048
	 - Recall: 0.4444
	 - MCC: 0.2390

	 - Train loss: 0.22920
	 - Accuracy: 0.8125
	 - F1 Score: 0.8090
	 - Precision: 0.7833
	 - Recall: 0.8786
	 - MCC: 0.7348

	 - Train loss: 0.22920
	 - Accuracy: 0.7500
	 - F1 Score: 0.8128
	 - Precision: 0.8042
	 - Recall: 0.8429
	 - MCC: 0.6515

	 - Train loss: 0.22920
	 - Accuracy: 0.5625
	 - F1 Score: 0.4008
	 - Precision: 0.4000
	 - Recall: 0.4167
	 - MCC: 0.2469

	 - Train loss: 0.22920
	 - Accuracy: 0.5000
	 - F1 Score: 0.5071
	 - Precision: 0.5905
	 - Recall: 0.6667
	 - MCC: 0.3419

	 - Train loss: 0.22920
	 - Accuracy: 0.7500
	 - F1 Score: 0.7595
	 - Precision: 0.7827
	 - Recall: 0.8036
	 - MCC: 0.6573

	 - Tra

Epoch: 100%|██████████| 8/8 [04:03<00:00, 30.42s/it]


	 - Train loss: 0.10739
	 - Accuracy: 0.7778
	 - F1 Score: 0.7778
	 - Precision: 0.7778
	 - Recall: 0.7778
	 - MCC: 0.5500


# Load the Fine Tuned Model

In [17]:
# load the model saved
PATH = './greet';
model = torch.load(PATH)

# Model Testing

Now let's go test our model on the portion of the Dataset that has been specially split for this purpose, in order to train our model on data it has never seen, again here we are going to print the model metrics for testing

In [19]:
for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    eval_output = model(b_input_ids, 
                        token_type_ids = None, 
                        attention_mask = b_input_mask)
    logits = eval_output.logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    test_predictions = np.argmax(logits, axis=1)
    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Accuracy: {:4f}'.format(accuracy.compute(predictions=test_predictions, references=label_ids)['accuracy']))
    print('\t - F1 Score: {:4f}'.format(f1.compute(predictions=test_predictions, references=label_ids, average="macro")['f1']))
    print('\t - Precision: {:4f}'.format(precision.compute(predictions=test_predictions, references=label_ids, average="macro")['precision']))
    print('\t - Recall: {:4f}'.format(recall.compute(predictions=test_predictions, references=label_ids, average="macro")['recall']))
    print('\t - MCC: {:.4f}'.format(mcc.compute(predictions=test_predictions, references=label_ids, average="macro")['matthews_correlation']))


	 - Train loss: 0.1074
	 - Accuracy: 0.500000
	 - F1 Score: 0.306818
	 - Precision: 0.277778
	 - Recall: 0.342857
	 - MCC: 0.2207

	 - Train loss: 0.1074
	 - Accuracy: 0.750000
	 - F1 Score: 0.857143
	 - Precision: 0.864583
	 - Recall: 0.864583
	 - MCC: 0.6104


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



	 - Train loss: 0.1074
	 - Accuracy: 0.750000
	 - F1 Score: 0.745238
	 - Precision: 0.700000
	 - Recall: 0.856250
	 - MCC: 0.6464

	 - Train loss: 0.1074
	 - Accuracy: 0.562500
	 - F1 Score: 0.563725
	 - Precision: 0.532738
	 - Recall: 0.666667
	 - MCC: 0.3459

	 - Train loss: 0.1074
	 - Accuracy: 0.875000
	 - F1 Score: 0.852381
	 - Precision: 0.855556
	 - Recall: 0.883333
	 - MCC: 0.7737

	 - Train loss: 0.1074
	 - Accuracy: 0.750000
	 - F1 Score: 0.735450
	 - Precision: 0.716667
	 - Recall: 0.878788
	 - MCC: 0.6535

	 - Train loss: 0.1074
	 - Accuracy: 0.687500
	 - F1 Score: 0.643939
	 - Precision: 0.638889
	 - Recall: 0.666667
	 - MCC: 0.5484

	 - Train loss: 0.1074
	 - Accuracy: 0.750000
	 - F1 Score: 0.701389
	 - Precision: 0.664286
	 - Recall: 0.791667
	 - MCC: 0.6292

	 - Train loss: 0.1074
	 - Accuracy: 0.812500
	 - F1 Score: 0.783333
	 - Precision: 0.812500
	 - Recall: 0.839286
	 - MCC: 0.7370

	 - Train loss: 0.1074
	 - Accuracy: 0.937500
	 - F1 Score: 0.905797
	 - Precision

We rerun the testing on the same portion of data as before, but this time the data on which the model has made the prediction are printed on the screen, plus the prediction and the oracle for each data, so as to be able to verify on which portions of code the model has more problems and improve the dataset, so as to improve the performance of the model as well

In [20]:
correct = 0
wrong = 0

for index, test in enumerate(test_codes):
  encoding = preprocessing(test, tokenizer)
  predict_ids = []
  predict_attention_mask = []
  # Extract IDs and Attention Mask
  predict_ids.append(encoding['input_ids'])
  predict_attention_mask.append(encoding['attention_mask'])
  predict_ids = torch.cat(predict_ids, dim = 0)
  predict_attention_mask = torch.cat(predict_attention_mask, dim = 0)

  # Forward pass, calculate logit predictions
  with torch.no_grad():
    output = model(predict_ids.to(device), token_type_ids = None, attention_mask = predict_attention_mask.to(device))
  # print("0 = method opposite comment; 1 = attribute opposite comment; 2 = clear")
  print(output.logits.softmax(dim=-1).tolist())
  prediction = np.argmax(output.logits.cpu().numpy()).flatten().item()
  print(test)
  if prediction == 0:
    print('predicted: mop');
  elif prediction == 1:
    print('predicted: aop');
  elif prediction == 2:
    print('predicted: clr');
  elif prediction == 3:
    print('predicted: nic');


  oracle = test_labels.numpy()[index]
  if oracle == 0:
    print('oracle: mop');
  elif oracle == 1:
    print('oracle: aop');
  elif oracle == 2:
    print('oracle: clr');
  elif oracle == 3:
    print('oracle: nic');

  if prediction == oracle:
    print("PASS")
    correct += 1
  else:
    print("FAULT")
    wrong += 1
  
  print("=================================================================================================================")

print("correct predictions: " + str(correct))
print("wrong predictions: " + str(wrong))


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


[[0.9686018228530884, 0.0010602999245747924, 0.02949131466448307, 0.0008466365397907794]]
def work_on_condition()
	"""
		this function do nothing for a condition
	"""
predicted: mop
oracle: mop
PASS
[[0.0017796080792322755, 0.001719863386824727, 0.9626646637916565, 0.033835917711257935]]
"""
	the function generates wordcloud on a certain text or file, checks to see if the data passed as a parameter is an instance of a string or file type, if not, throws an exception
     Args:
         data: the data from which to generate wordcloud
"""
def get_word(data):
    from wordcloud import WordCloud, STOPWORDS
    import matplotlib.pyplot as plt
    stopwords = set(STOPWORDS)
    if os.path.isfile(data):
        with open(data, 'r') as f:
            data = f.read()

    data = ' '.join(data.lower().split(' '))
    wordcloud = WordCloud(width=400, height=400,
                          background_color='white',
                          stopwords=stopwords,
                          min_font_si